In [7]:
!pip install -q tensorflow-recommenders
!pip install -q scann

In [6]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

from sklearn.model_selection import train_test_split

In [21]:
# this is just a simulation, the real data must be obtain by a call to the database api
# returns all the activities available in that slot!
activities = [
    {'id': 'A1', 'name': 'football', 'class': 'sports'},
    {'id': 'A2', 'name': 'chess', 'class': 'boardgame'},
    {'id': 'B1', 'name': 'jenga', 'class': 'boardgame'},
    {'id': 'C1', 'name': 'salsa', 'class': 'dancing'}
    # .... all activities
]

In [22]:
# todo: see if its ok to do this computation here or should it be another request to db
unique_classes = set(activity['class'] for activity in activities)
unique_classes

{'boardgame', 'dancing', 'sports'}

In our retrieval system the user acts as implicit feedback to the model since:

*   Choosing the activity is a positive feedback
*   Not choosing an activity is a negative feedback

It is not explicit since they don't tell us **how much** they like the activity.


In [31]:
# Implicit feedback: counter of the times the user has picked or not the activity
# picked        +1
# not picked    +0
user_feedback = {
    1:{'A1': 2, 'A2': 0, 'B1': 1,'C1': 0 }, # User 1 picked activity A1 twice (positive feedback). but has never picked activity A2 (negative feedback),...
    2:{'A1': 1, 'A2': 0, 'B1': 0,'C1': 1 },
    3:{'A1': 0, 'A2': 0, 'B1': 1,'C1': 0 },
    # Todo: Append feedback when recieving
}

In [32]:
# Create X and y for each user
X_list = []
y_list = []
for user_id in user_feedback:
    user_activities = user_feedback[user_id]
    user_vector = np.array([1 if activity['id'] in [activity_id for activity_id, _ in user_activities] else 0 for activity in activities], dtype=np.float32)
    feedback_vector = np.array([feedback for _, feedback in user_activities], dtype=np.float32)

    X_list.append(user_vector)
    y_list.append(feedback_vector)

X = np.array(X_list)
y = np.array(y_list)

In [36]:
X

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)

In [37]:
y

array([[1., 2., 1., 1.],
       [1., 2., 1., 1.],
       [1., 2., 1., 1.]], dtype=float32)

## Define and compile model

In [34]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [35]:
# train the model for each user
for i in range(len(X)):
    model.fit(X[i], y[i], epochs=50, batch_size=1, verbose=1)

Epoch 1/50


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(1,), dtype=float32). Expected shape (None, 4), but input has incompatible shape (1,)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1,), dtype=float32)
  • training=True
  • mask=None

In [ ]:
def recommend_activities(user_id, model, activities):
    user_vector = np.array([1 if activity['id'] in [activity_id for activity_id, _ in user_feedback_dict[user_id]] else 0 for activity in activities], dtype=np.float32)

    # Predecir el feedback implícito para actividades no realizadas por el usuario
    predictions = model.predict(user_vector.reshape(1, -1)).flatten()

    # Ordenar actividades por las predicciones (mayor probabilidad de feedback positivo primero)
    sorted_indices = np.argsort(predictions)[::-1][:5]
    recommended_activities = [activities[i]['name'] for i in sorted_indices]

    return recommended_activities



In [ ]:
# Generar recomendaciones para un usuario específico (por ejemplo, usuario ID 1)
user_id = 1
recommended_activities = recommend_activities(user_id, model, activities)

print(f"Recomendaciones para Usuario {user_id}:")
for activity in recommended_activities:
    print(activity)

Wait

In [27]:
# Represent activities as a one-hot encoding vectors
def activity_vector(activity):
    vec_act = []
    for class_i in unique_classes:
        vec_act.append(1 if activity['class'] == class_i else 0)
    return vec_act

# Create matrices X (activity features) and y (user preferences)
X = np.array([activity_vector(activity) for activity in activities], dtype=np.float32)
y = np.array([feedback[2] for feedback in user_feedback], dtype=np.float32)

In [20]:
X

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)

In [26]:
y

array([1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0.], dtype=float32)